## Natural Language to SQL using Google's Gemini Pro

<a href="https://colab.research.google.com/github/bhattbhavesh91/n2sql-google-gemini/blob/main/n2sql-google-gemini-notebook.ipynb" target="_blank"><img height="40" alt="Run your own notebook in Colab" src = "https://colab.research.google.com/assets/colab-badge.svg"></a>

# Installation

In [1]:
!pip install -q google-generativeai==0.3.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.6/146.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.7/598.7 kB 12.2 MB/s eta 0:00:00


# Imports

In [16]:
# !pip install --upgrade langchain sqlalchemy langchain_openai

In [17]:
import google.generativeai as genai
from pathlib import Path
import sqlite3
import pandas as pd
from langchain_community.agent_toolkits.sql.base import create_sql_agent
from langchain_openai import ChatOpenAI
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

# Convert pandas to sql db

In [99]:
df = pd.read_csv("archive.zip")
df.head()

,index,url,name,sku,selling_price,original_price,currency,availability,color,category,...,source_website,breadcrumbs,description,brand,images,country,language,average_rating,reviews_count,crawled_at
0,0,https://www.adidas.com/us/beach-shorts/FJ5089....,Beach Shorts,FJ5089,40,NaN,USD,InStock,Black,Clothing,...,https://www.adidas.com,Women/Clothing,Splashing in the surf. Making memories with yo...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.5,35,2021-10-23 17:50:17.331255
1,1,https://www.adidas.com/us/five-ten-kestrel-lac...,Five Ten Kestrel Lace Mountain Bike Shoes,BC0770,150,NaN,USD,InStock,Grey,Shoes,...,https://www.adidas.com,Women/Shoes,Lace up and get after it. The Five Ten Kestrel...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.8,4,2021-10-23 17:50:17.423830
2,2,https://www.adidas.com/us/mexico-away-jersey/G...,Mexico Away Jersey,GC7946,70,NaN,USD,InStock,White,Clothing,...,https://www.adidas.com,Kids/Clothing,"Clean and crisp, this adidas Mexico Away Jerse...",adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.9,42,2021-10-23 17:50:17.530834
3,3,https://www.adidas.com/us/five-ten-hiangle-pro...,Five Ten Hiangle Pro Competition Climbing Shoes,FV4744,160,NaN,USD,InStock,Black,Shoes,...,https://www.adidas.com,Five Ten/Shoes,The Hiangle Pro takes on the classic shape of ...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,3.7,7,2021-10-23 17:50:17.615054
4,4,https://www.adidas.com/us/mesh-broken-stripe-p...,Mesh Broken-Stripe Polo Shirt,GM0239,65,NaN,USD,InStock,Blue,Clothing,...,https://www.adidas.com,Men/Clothing,Step up to the tee relaxed. This adidas golf p...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.7,11,2021-10-23 17:50:17.702680


In [27]:
df.shape

(845, 21)

In [31]:
df.columns

Index(['index', 'url', 'name', 'sku', 'selling_price', 'original_price',
       'currency', 'availability', 'color', 'category', 'source',
       'source_website', 'breadcrumbs', 'description', 'brand', 'images',
       'country', 'language', 'average_rating', 'reviews_count', 'crawled_at'],
      dtype='object')

In [100]:
df_usd = df[(df['currency'] == 'USD') & (df['selling_price'] > 0) & (df['language'] == 'en')]
df_usd.shape
df.shape

(845, 21)

In [101]:
df = df.drop(columns=['index','url', 'crawled_at','original_price','images','language','source','source_website','country','currency'], axis=1)

In [61]:
df.shape

(845, 12)

In [102]:
# Rename columns
df = df.rename(columns={
    'name': 'product_name',
    'selling_price': 'price',
    'brand': 'brand_name',
    'description': 'product_desc',
    'category': 'category_name',
    'sku': 'product_id'
})
df.columns


Index(['product_name', 'product_id', 'price', 'availability', 'color',
       'category_name', 'breadcrumbs', 'product_desc', 'brand_name',
       'average_rating', 'reviews_count'],
      dtype='object')

In [103]:
conn = sqlite3.connect('fashion_db.sqlite')
c = conn.cursor()

c.execute('CREATE TABLE IF NOT EXISTS fashion_products (product_name text, brand_name text, category_name text, price int, average_rating float,\
 color text, product_desc text, reviews_count int, availlability text, breadcrumbs text, product_id text)')
conn.commit()

df.to_sql('fashion_products', conn, if_exists='replace', index = False)


845

In [104]:
c.execute('''
SELECT product_name FROM fashion_products LIMIT 10
          ''')

for row in c.fetchall():
    print (row)
c.close()

('Beach Shorts',)
('Five Ten Kestrel Lace Mountain Bike Shoes',)
('Mexico Away Jersey',)
('Five Ten Hiangle Pro Competition Climbing Shoes',)
('Mesh Broken-Stripe Polo Shirt',)
('EQT Spikeless Golf Shoes',)
('Adicross Hybrid Shorts',)
('Tiro 21 Windbreaker',)
('Classic 3-Stripes Swimsuit',)
('Tiro 21 Windbreaker',)


# SQL Query Executor

In [73]:
def read_sql_query(sql, db):
    conn = sqlite3.connect(db)
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    for row in rows:
        print(row)
    conn.close()

In [105]:
def get_row_count(db_name):
    """Generates SQL query to find total number of rows in a database table.

    Args:
        db_name: Name of the SQLite database file.

    Returns:
        SQL query as a string.
    """

    return "SELECT COUNT(*) FROM fashion_products;"



In [106]:
sql_query = get_row_count('fashion_products')
sql_query
# To execute the query use the read_sql_query function from previous code.
read_sql_query(sql_query, 'fashion_db.sqlite')

(845,)


In [107]:
read_sql_query('SELECT * FROM fashion_products LIMIT 10;', 'fashion_db.sqlite')

('Beach Shorts', 'FJ5089', 40, 'InStock', 'Black', 'Clothing', 'Women/Clothing', 'Splashing in the surf. Making memories with your friends. Beach days are the best days. These shorts are made of stretchy woven fabric. An elastic waistband that features the adidas logo brings a sporty look to your day at the beach.', 'adidas', 4.5, 35)
('Five Ten Kestrel Lace Mountain Bike Shoes', 'BC0770', 150, 'InStock', 'Grey', 'Shoes', 'Women/Shoes', "Lace up and get after it. The Five Ten Kestrel Lace Mountain Bike Shoes offer efficient pedal power with low-profile style. The wide platform is compatible with all clipless pedals and offers high-friction grip on and off the bike. You'll find the find comfort and versatility for extended trail rides and afterwork hot laps alike.", 'adidas', 4.8, 4)
('Mexico Away Jersey', 'GC7946', 70, 'InStock', 'White', 'Clothing', 'Kids/Clothing', "Clean and crisp, this adidas Mexico Away Jersey for juniors displays a design that began life in an artist's studio. So

#Using OpenAI LLM and LangChain Agents

In [27]:
api_key = userdata.get('OPENAI_API_KEY')

In [28]:
agent = create_sql_agent(llm = ChatOpenAI(model = "gpt-4o-mini", api_key=api_key),
                        agent_type = "openai-tools",
                        toolkit = SQLDatabaseToolkit(db = SQLDatabase.from_uri(database_uri = "sqlite:///fashion_db.sqlite"),
                                                     llm = ChatOpenAI(model = "gpt-4o-mini", api_key=api_key)))


In [29]:
agent.invoke({"input": "How many rows are in the data?"})

{'input': 'How many rows are in the data?',
 'output': 'There are 845 rows in the "fashion_products" table.'}

In [30]:
agent.invoke({"input": "How many products of Nike are there?"})

{'input': 'How many products of Nike are there?',
 'output': 'There are no products of Nike in the database.'}

In [31]:
agent.invoke({'input': 'What is the product id and product name of the most expensive product in the database?'})

{'input': 'What is the product id and product name of the most expensive product in the database?',
 'output': 'The most expensive product in the database is the "Terrex Primeknit Rain.RDY Jacket" with the product ID "GI7309".'}

In [32]:
def most_expensive_product(db_name):
    conn = sqlite3.connect(db_name)
    c = conn.cursor()
    c.execute('''
    SELECT product_id, product_name FROM fashion_products ORDER BY price DESC LIMIT 1;
    ''')
    result = c.fetchone()
    conn.close()
    return result

most_expensive_product('fashion_db.sqlite')


('GI7309', 'Terrex Primeknit Rain.RDY Jacket')

#Using Google Gemini LLM

In [ ]:
from google.colab import userdata
genai.configure(api_key = userdata.get('GEMINI_KEY'))

In [ ]:
genai.__version__

In [ ]:
# Set up the model
generation_config = {
  "temperature": 0.4,
  "top_p": 1,
  "top_k": 32,
  "max_output_tokens": 4096,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  }
]

In [ ]:
model = genai.GenerativeModel(model_name = "gemini-pro",
                              generation_config = generation_config,
                              safety_settings = safety_settings)

# Define Prompt

In [90]:
df.columns

Index(['product_name', 'product_id', 'price', 'availability', 'color',
       'category_name', 'breadcrumbs', 'product_desc', 'brand_name',
       'average_rating', 'reviews_count'],
      dtype='object')

In [91]:
prompt_parts_1 = [
  "You are an expert in converting English questions to SQL code! The SQL database has the name fashion_products and has the following columns - product_id, product_name, brand_name, category_name, price, color, product_desc, average_rating, review_counts, breadcrumbs, and availability.\n\nFor example,\nExample 1 - How many entries of Adidas are present?, the SQL command will be something like this\n``` SELECT COUNT(*) FROM fashion_products WHERE brand = 'Adidas';\n```\n\nExample 2 - How many XL products of Nike are there that have a rating of more than 4?\n```\nSELECT COUNT(*) FROM fashion_products WHERE brand = 'Nike' AND size = 'XL' AND \"Rating\" > 4;\n```\n\nExample 3 - \n```\nSELECT product_name FROM fashion_products WHERE price = (SELECT MAX(price) FROM fashion_products);\n```\n\nDont include ``` and \\n in the output",
]

In [108]:
question = "Tell me the id of the most expensive T-shirt?"

In [109]:
prompt_parts = [prompt_parts_1[0], question]
response = model.generate_content(prompt_parts)
response.text

"```SELECT product_id FROM fashion_products WHERE product_name LIKE '%T-shirt%' AND price = (SELECT MAX(price) FROM fashion_products);```"

In [111]:
read_sql_query('SELECT product_id FROM fashion_products WHERE product_name LIKE "%T-shirt%" AND price = (SELECT MAX(price) FROM fashion_products);',
               "fashion_db.sqlite")

# Combine it into Function

In [112]:
def generate_gemini_response(question, input_prompt):
    prompt_parts = [input_prompt, question]
    response = model.generate_content(prompt_parts)
    output = read_sql_query(response.text, "fashion_db.sqlite")
    return output

In [115]:
generate_gemini_response("How many products of Nike are there?",
                         prompt_parts_1[0])

(0,)
